# 模型初始化
在深度学习模型的训练中，权重的初始值极为重要。一个好的初始值，会使模型收敛速度提高，使模型准确率更精确。一般情况下，我们不使用全0初始值训练网络。为了利于训练和减少收敛时间，我们需要对模型进行合理的初始化。PyTorch也在torch.nn.init中为我们提供了常用的初始化方法

## torch.nn.init使用
通常会根据实际模型来使用torch.nn.init进行初始化，通常使用isinstance()来进行判断模块属于什么类型。

In [5]:
import torch
import torch.nn as nn

conv = nn.Conv2d(1, 3, 3)
linear = nn.Linear(10, 1)

print(isinstance(conv,nn.Conv2d)) # 判断conv是否是nn.Conv2d类型
print(isinstance(linear,nn.Conv2d)) # 判断linear是否是nn.Conv2d类型

True
False


In [6]:
# 查看随机初始化的conv参数
print(conv.weight.data)
# 查看linear的参数
print(linear.weight.data)

tensor([[[[ 0.0649,  0.0499,  0.2690],
          [-0.2581, -0.2955,  0.1092],
          [ 0.1342,  0.0149, -0.1022]]],


        [[[ 0.1881,  0.2658,  0.2167],
          [ 0.1855,  0.1850, -0.0031],
          [ 0.2147, -0.3313,  0.2925]]],


        [[[ 0.1777, -0.2966,  0.1964],
          [-0.1386,  0.0648, -0.0646],
          [ 0.0847,  0.1620, -0.1190]]]])
tensor([[-0.1683,  0.1213,  0.0343,  0.1980, -0.0388,  0.2920,  0.1678,  0.1059,
         -0.2815,  0.0879]])


In [7]:
# 对conv进行kaiming初始化
torch.nn.init.kaiming_normal_(conv.weight.data)
print(conv.weight.data)
# 对linear进行常数初始化
torch.nn.init.constant_(linear.weight.data,0.3)
print(linear.weight.data)

tensor([[[[ 0.5562, -0.2481,  0.3210],
          [-0.1728, -0.4981,  0.7003],
          [ 0.6046, -0.2711,  0.3497]]],


        [[[-0.1609, -0.5650,  0.3399],
          [ 0.1784, -0.3683,  0.0305],
          [-0.1990, -0.1506, -1.0631]]],


        [[[ 0.1771,  0.8817, -0.2890],
          [ 0.2547,  0.2746,  0.1981],
          [-0.6396, -0.1469,  0.1378]]]])
tensor([[0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000, 0.3000,
         0.3000]])


## 初始化函数的封装
将各种初始化方法定义为一个initialize_weights()的函数并在模型初始后进行使用

In [8]:
"""
遍历当前模型的每一层，然后判断各层属于什么类型，
然后根据不同类型层，设定不同的权值初始化方法
"""
def initialize_weights(model):
	for m in model.modules():
		# 判断是否属于Conv2d
		if isinstance(m, nn.Conv2d):
			torch.nn.init.zeros_(m.weight.data)
			# 判断是否有偏置
			if m.bias is not None:
				torch.nn.init.constant_(m.bias.data,0.3)
		elif isinstance(m, nn.Linear):
			torch.nn.init.normal_(m.weight.data, 0.1)
			if m.bias is not None:
				torch.nn.init.zeros_(m.bias.data)
		elif isinstance(m, nn.BatchNorm2d):
			m.weight.data.fill_(1) 		 
			m.bias.data.zeros_()

使用初始化函数封装

In [9]:
# 模型的定义
class MLP(nn.Module):
  # 声明带有模型参数的层，这里声明了两个全连接层
  def __init__(self, **kwargs):
    # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Conv2d(1,1,3)
    self.act = nn.ReLU()
    self.output = nn.Linear(10,1)
    
   # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
  def forward(self, x):
    o = self.act(self.hidden(x))
    return self.output(o)

mlp = MLP()
print(mlp.hidden.weight.data)
print("-------初始化-------")

mlp.apply(initialize_weights) # 或者initialize_weights(mlp)
print(mlp.hidden.weight.data)

tensor([[[[-0.1400, -0.1466, -0.2628],
          [ 0.3193,  0.2986,  0.2296],
          [-0.2773,  0.1935, -0.2867]]]])
-------初始化-------
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])


注意： 我们在初始化时，最好不要将模型的参数初始化为0，因为这样会导致梯度消失，从而影响模型的训练效果。因此，我们在初始化时，可以使用其他初始化方法或者将模型初始化为一个很小的值，如0.01，0.1等。